In [1]:
library(tidyverse)
library(yaml)
library(rstan)
library(tidybayes)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: StanHeaders


rstan version 2.32.3 (Stan version 2.26.1)


For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)
For within-chain threading using `reduce_sum()` or `map_rect()` Stan functions,
chan

## Config and data

In [2]:
config_id <- "pooling_sim_no_pool"
data_config_id <- "pooling_sim"

In [3]:
data_config <- yaml.load_file(paste0("../data/configs/", data_config_id, "/data.yaml"))
main_config <- yaml.load_file(paste0("../experiments/configs/", config_id, "/main.yaml"))

In [4]:
results_base_dir <- paste0("../experiments/results/", config_id)
if (!dir.exists(results_base_dir)) {
    dir.create(results_base_dir, recursive = TRUE)
}

### Data

In [5]:
data_base_dir <- paste0("../", data_config$output_dir)
data_path <- paste0(data_base_dir, "/data.csv")
beta_0 <- data_config$beta_0
beta_1 <- data_config$beta_1

In [6]:
data_df <- read_csv(data_path)

Rows: 750 Columns: 7
── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
dbl (7): group_id, group_effect, x, indiv_effect, indiv_id, measurement_erro...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


### Model

In [7]:
model_seed <- main_config$seed
n_samp <- main_config$n_samp
n_warmup_samp <- main_config$n_warmup_samp
n_chains <- main_config$n_chains
model_file_path <- paste0("../", main_config$model_file_path)

## Model fitting

In [8]:
stan_data <- list(
    N = data_df %>% nrow(),
    n_indiv = data_df %>% distinct(indiv_id) %>% nrow(),
    y = data_df %>% pull(y),
    x = data_df %>% pull(x),
    indiv_id = data_df %>% pull(indiv_id)
)

In [9]:
beta_0_names <- paste0("beta_0[", 1:stan_data$n_indiv, "]")
beta_1_names <- paste0("beta_1[", 1:stan_data$n_indiv, "]")

In [10]:
if (!file.exists(paste0(results_base_dir, "/fit.rds"))) {
    fit <- stan(
        file = model_file_path,
        data = stan_data,
        chains = n_chains,
        iter = n_samp,
        warmup = n_warmup_samp,
        seed = model_seed,
        cores = n_chains,
        # pars = c(beta_0_names, beta_1_names),
        pars = c("beta_0", "beta_1"),
        include = TRUE
    )
    saveRDS(fit, paste0(results_base_dir, "/fit.rds"))
    
    tidy_draws_df <- fit %>% tidybayes::spread_draws(beta_0[i], beta_1[i])
    tidy_draws_df %>%
        write_csv(paste0(results_base_dir, "/draws.csv"))
} else {
    fit <- readRDS(paste0(results_base_dir, "/fit.rds"))
    tidy_draws_df <- read_csv(paste0(results_base_dir, "/draws.csv"))
}

Warning message:
“There were 733 transitions after warmup that exceeded the maximum treedepth. Increase max_treedepth above 10. See
https://mc-stan.org/misc/warnings.html#maximum-treedepth-exceeded”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”


In [11]:
tidy_draws_long_df <- tidy_draws_df %>%
    pivot_longer(cols = !c(".chain", ".iteration", ".draw"), names_to = "param")

In [12]:
fit_summary <- summary(fit)

In [13]:
fit_summary$summary %>%
    as_tibble(rownames="param")

param,mean,se_mean,sd,2.5%,25%,50%,75%,97.5%,n_eff,Rhat
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
beta_0[1],50.11936,0.1716870,9.870053,30.91295,43.30288,50.05061,56.93268,68.92037,3304.943,1.0014159
beta_0[2],50.05914,0.1740984,9.920573,30.82301,43.41648,50.01690,56.55366,70.40511,3247.014,1.0000631
beta_0[3],50.01464,0.1709642,10.016901,30.10208,43.51953,49.85464,56.52045,69.72601,3432.864,0.9998387
beta_0[4],49.83387,0.1825420,10.145745,29.93735,43.17271,49.95326,56.50535,69.39405,3089.175,1.0001202
beta_0[5],49.85014,0.1738992,10.272068,30.07610,42.89269,49.77458,56.89790,70.10488,3489.158,1.0005286
beta_0[6],49.84506,0.1627593,10.109258,30.43329,42.81967,49.77955,56.83972,69.28644,3857.864,0.9995736
beta_0[7],49.96278,0.1812034,9.980025,30.47987,43.29968,50.04126,56.59542,69.16809,3033.407,0.9999800
beta_0[8],49.86133,0.1688214,10.079511,30.36228,42.81638,49.98488,56.75329,69.73460,3564.709,0.9995802
beta_0[9],50.02931,0.1832270,10.442514,29.69415,42.91416,50.00072,57.20557,70.67494,3248.112,1.0001935
